In [10]:
import garner

In [2]:
garner.login("kipgparker@gmail.com")

Enter Password


Password: ·········


Authenticated


In [4]:
res = garner.select_pool(pool_name='This pool is a test')

In [3]:
ws = garner.webSocket

ws.attach_auth(garner.auth)

ws.connect("6ce54ac7-2761-4916-971c-c6286ffde71e")

connected
>> {"type": "connection_init"}
{"id":"e80cb5db-788a-4a84-9604-513cb3e8152b","type":"data","payload":{"data":{"onSampleCompleted":{"poolID":"6ce54ac7-2761-4916-971c-c6286ffde71e","id":"e4a715ad-14de-4ddc-92e5-9503ae92cec2","x":["x"],"y":["y2"]}}}}


In [4]:
ws.disconnect()